# Ray RLlib 강화학습 실험

이 노트북은 Ray RLlib을 사용한 강화학습 연구의 기본 템플릿입니다.

In [ ]:
# 필수 라이브러리 import
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.algorithm import Algorithm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os

# 결과 저장 디렉토리 설정
RESULTS_DIR = Path("../results")
RESULTS_DIR.mkdir(exist_ok=True)

print("라이브러리 import 완료")

## 1. Ray 초기화

In [ ]:
# Ray 초기화
# 로컬 모드로 실행 (디버깅용)
# ray.init(local_mode=True)

# 일반 모드로 실행 (실험용)
ray.init(ignore_reinit_error=True)

print(f"Ray 클러스터 정보: {ray.cluster_resources()}")

## 2. 환경 설정

**참고**: 실제 환경은 별도 프로젝트로 관리되므로, 여기서는 예시로 Gymnasium 환경을 사용합니다.
실제 환경을 사용할 때는 해당 환경의 import 경로를 수정하세요.

In [ ]:
# 환경 설정
# TODO: 실제 환경으로 교체
# from your_env_project import YourEnv

# 예시: Gymnasium 환경 사용
ENV_NAME = "CartPole-v1"  # 실제 환경으로 교체

# 환경 정보 확인
import gymnasium as gym
env = gym.make(ENV_NAME)
print(f"환경: {ENV_NAME}")
print(f"관찰 공간: {env.observation_space}")
print(f"행동 공간: {env.action_space}")
env.close()

## 3. RLlib 알고리즘 설정

In [ ]:
# PPO 알고리즘 설정 (다른 알고리즘으로 변경 가능: DQN, A3C, SAC 등)
config = (
    PPOConfig()
    .environment(env=ENV_NAME)
    .training(
        lr=3e-4,
        train_batch_size=4000,
        sgd_minibatch_size=128,
        num_sgd_iter=10,
    )
    .resources(num_gpus=0)  # GPU 사용 시 1로 변경
    .rollouts(num_rollout_workers=2)  # 병렬 워커 수
    .framework("torch")  # "tf2" 또는 "torch"
)

print("알고리즘 설정 완료")
print(config)

## 4. 알고리즘 인스턴스 생성 및 학습

In [ ]:
# 알고리즘 인스턴스 생성
algo = config.build()

print("알고리즘 인스턴스 생성 완료")

In [ ]:
# 학습 실행
NUM_ITERATIONS = 100  # 학습 반복 횟수
training_history = []

for i in range(NUM_ITERATIONS):
    result = algo.train()
    training_history.append({
        'iteration': i + 1,
        'episode_reward_mean': result.get('episode_reward_mean', 0),
        'episode_len_mean': result.get('episode_len_mean', 0),
        'policy_loss': result.get('info', {}).get('learner', {}).get('default_policy', {}).get('policy_loss', 0),
    })
    
    if (i + 1) % 10 == 0:
        print(f"Iteration {i + 1}: Mean Reward = {result.get('episode_reward_mean', 0):.2f}")

print("학습 완료")

## 5. 학습 결과 시각화

In [ ]:
# 학습 히스토리를 DataFrame으로 변환
df = pd.DataFrame(training_history)

# 결과 저장
df.to_csv(RESULTS_DIR / "training_history.csv", index=False)
print(f"결과 저장 완료: {RESULTS_DIR / 'training_history.csv'}")

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 에피소드 평균 보상
axes[0].plot(df['iteration'], df['episode_reward_mean'])
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Mean Episode Reward')
axes[0].set_title('Training Progress: Mean Reward')
axes[0].grid(True)

# 에피소드 평균 길이
axes[1].plot(df['iteration'], df['episode_len_mean'])
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Mean Episode Length')
axes[1].set_title('Training Progress: Mean Episode Length')
axes[1].grid(True)

plt.tight_layout()
plt.savefig(RESULTS_DIR / "training_curves.png", dpi=150)
plt.show()

print(f"시각화 저장 완료: {RESULTS_DIR / 'training_curves.png'}")

In [ ]:
# 학습된 정책으로 평가
NUM_EVAL_EPISODES = 10
eval_results = []

env = gym.make(ENV_NAME, render_mode=None)

for episode in range(NUM_EVAL_EPISODES):
    obs, info = env.reset()
    episode_reward = 0
    episode_length = 0
    done = False
    
    while not done:
        action = algo.compute_single_action(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        episode_reward += reward
        episode_length += 1
    
    eval_results.append({
        'episode': episode + 1,
        'reward': episode_reward,
        'length': episode_length
    })

env.close()

eval_df = pd.DataFrame(eval_results)
print("\n평가 결과:")
print(eval_df)
print(f"\n평균 보상: {eval_df['reward'].mean():.2f} ± {eval_df['reward'].std():.2f}")
print(f"평균 에피소드 길이: {eval_df['length'].mean():.2f} ± {eval_df['length'].std():.2f}")

# 평가 결과 저장
eval_df.to_csv(RESULTS_DIR / "evaluation_results.csv", index=False)

## 7. 모델 저장 및 로드

In [ ]:
# 모델 저장
checkpoint_dir = algo.save(RESULTS_DIR / "checkpoint")
print(f"모델 저장 완료: {checkpoint_dir}")

# 모델 로드 예시 (주석 처리)
# loaded_algo = Algorithm.from_checkpoint(checkpoint_dir)

## 8. 정리

In [ ]:
# 알고리즘 정리
algo.stop()

# Ray 종료 (필요한 경우)
# ray.shutdown()

print("정리 완료")